# 阅读笔记

** 作者：方跃文 **

** Email: fyuewen@gmail.com **

** 时间：始于2018年3月6日， 结束写作于 2018年7月**






# 第六章 数据加载、存储和文件格式

数据如果不能导入和导出，那么本书中介绍的工具自然也就没有用武之地了。在我们的日常生活和研究中，有许多文件输入和输出的应用场景。

输入和输出通常划分为几个大类：读取文本文件和其他更高效的磁盘存储格式，加载数据库中的数据，利用web api操作网络资源。

## 读写文本格式的数据

pandas 提供了一些用于将表格型数据读取为 DataFrame 对象的函数。表6-1对此进行了总结，其中 read_csv and read_table 将会是我们经常用到的。

Table 6-1. Parsing functions in pandas

| Functions       | Description|
| ------------- |:-------------:|
| read_csv      | load delimited data from a file, URL, or file-like object; use comma as defult delimiter | 
| read_table | load delimited data from a file, URL, or file-like object; use tab ('\t') as defult delimiter |
| read_fwf | read data in fixed-width column format, i.e. no delimiters|
|read_clipboard| Version of read_table that reads data from the clipboard; useful for converting tables from web pages|
|read_excel | read tabular data from an Excel XLS or XLSX file |
| read_hdf | read **HDF5** files **written by pandas** |
| read_html | read all tables found in a given HTML document|
| read_json | read data from a JSON |
| **read_msgpack** | read pandas data encoded using the MessagePack binary format|
| **read_pickle** | read an arbitrary object in Python pickle format|  pickle竟然是咸菜的意思哦～
|read_sas| read a SASdataset stored in one of the SAS system's custom strorage format|
| **read_sql** | read the results of a SQL query (using SQLAlchemy) asa pandas DataFrame|
| read_stata | read a dataset from Stata file format |
| read_feather | read the Feather binary file format |

上述函数的重要功能之一就是type inferende，也就是类型推断。我们不需要为所读入的数据指定是什么类型。不过日期和其他自定义类型数据的处理
需要小心一些，因为自定义数据可能并不那么容易被推断出来属于何种数据类型。

HDF5，Feather和mgspack在文件中就会有数据类型，因此读入会更加方便。

Let's start with a small comma-separated csv file:


In [7]:
!cat chapter06/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

由于这个是逗号隔开的文本，我们可以方便地使用 read_csv 来读入数据

In [9]:
import pandas as pd

df = pd.read_csv('chapter06/ex1.csv')


In [10]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
当然我们也可以使用 read_table，只需要将delimiter指定为逗号即可

In [11]:
import pandas as pd

df1 = pd.read_table('chapter06/ex1.csv', sep=',')

In [12]:
df1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


上面这个例子中是已经包含了header部分了，但并不是所有数据都自带header，例如

In [15]:
import pandas as pd

df1 = pd.read_table('chapter06/ex2.csv', sep=',')
df2 = pd.read_table('chapter06/ex2.csv', sep=',', header=None)

In [14]:
df1

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [16]:
df2

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


df1中我没有指明是header=None，所以原始文件中都第一行被错误地用作了header。为了
避免这种错误，需要明确指明header=None

当然，我们也可以自己去指定名字：

In [18]:
import pandas as pd

df1 = pd.read_csv('chapter06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])
df1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


如果我们在读入数据都时候，就希望其中都某一列作为索引index，比如我们可以对上面这个例子，将message这列作为索引：

In [21]:
import pandas as pd

names = ['a', 'b', 'c', 'd', 'message']
df1 = pd.read_csv('chapter06/ex2.csv', names=names, index_col='message')

In [22]:
df1

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


如果我们想从多列转化成层次化索引（hierarchical index），我们需要由列编号或者列明组成都列表即可。

先看个例子

In [23]:
!cat ./chapter06/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [24]:
import pandas as pd

df = pd.read_csv('chapter06/csv_mindex.csv', index_col=['key1', 'key2'])

In [25]:
df

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

实际情况中，有的表格的分隔符并不是逗号，比如还有空白符等。对于这种情形，
我们可以传递一个正则表达式作为分隔符。例如对于下面这个例子，我们就可以使用
正则表达式 \s+ 来表示；

In [26]:
!head chapter06/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [27]:
list(open('chapter06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

从上面这两个linux命令可以看出，这个数据中的分隔符采用了空格。

In [29]:
import pandas as pd

result = pd.read_table('chapter06/ex3.txt', sep='\s+')

In [30]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


The parser functions（解析器函数）in the Table 6-1 have many aiddtional arguments to help
handle the wide variaty of exception fil formats that occur.让我来举个例子。我们可以利用skiprows来
跳过文件中的前面几行或者特别想指定的行。这个功能是相当有用的。特别是像我这样的人，喜欢在rawdata里面
写几句comments来说明一下这些数据的内容。

In [31]:
!cat chapter06/ex4.csv


# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [32]:
import pandas as pd

df = pd.read_csv('./chapter06/ex4.csv', skiprows=[0,2,3])

In [33]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


在chapter05的笔记中，我也说到过，实际处理的数据是很有可能包含空数剧的。那些missing data通常
不会被呈现出来，或者会被一些sentinel value所替代，例如NA或NULL

In [34]:
!cat chapter06/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [36]:
import pandas as pd

result = pd.read_csv('./chapter06/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [37]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [38]:
pd.notnull(result)

,something,a,b,c,d,message
0,True,True,True,True,True,False
1,True,True,True,False,True,True
2,True,True,True,True,True,True


The na_values option can take either a list or a set of strings to consider
missing values:也就是说只要原始数据中出现了na_values所指定的字符串或者list，就会以
NaN的方式呈现在pandas数据中。

In [39]:
result = pd.read_csv('chapter06/ex5.csv', na_values = 'NULL') # 注意原始数据中有一处有连续两个逗号，那就是产生NULL的地方

In [40]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


可以用一个字典为各个列指定不同的NA标记值：

In [41]:
sentinels = {'message':['foo', 'NA'], 'something': ['two']}
pd.read_csv('chapter06/ex5.csv', na_values = sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading text files in pieces 逐块读取文本文件

对于很大的文件，我们一般希望只读取一小部分，或者对小部分进行迭代式地读入。

In [50]:
result = pd.read_csv('./chapter06/ex6.csv', nrows=5)  # 我们只从原始数据中读入5行

In [52]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


如果想逐块读入文件，我们可以指定chunksize来实现：

In [53]:
chunker = pd.read_csv('chapter06/ex6.csv', chunksize = 1000)

In [54]:
chunker

这里 TextParser 允许我们可以进行迭代。

In [57]:
chunker = pd.read_csv('chapter06/ex6.csv', chunksize = 1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

这样我们就有了

In [58]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

对 value_counts 有点忘记了，所以特意地写了下面这个简单对例子来说明。具体对可以在chapter05中看笔记。

In [75]:
result = pd.DataFrame([1,2,3,4,22,2,2,3,4])

In [78]:
result[0]

0     1
1     2
2     3
3     4
4    22
5     2
6     2
7     3
8     4
Name: 0, dtype: int64

In [77]:
result[0].value_counts()

2     3
4     2
3     2
22    1
1     1
Name: 0, dtype: int64

### Writing data to text format 将数据输出到文本

之前我们只介绍了读入文本，但是却没有说输出文本。先来看些简单都例子：

In [83]:
!cat chapter06/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [2]:
import pandas as pd

data = pd.read_csv('chapter06/ex5.csv')

In [3]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [4]:
data.to_csv('chapter06/out.csv')

In [82]:
!cat chapter06/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


相比于原始数据，我们注意到输出的csv文件中也包含了索引。

In [6]:
import sys  #使用了sys.out 所以data会被直接打印在屏幕端，而不是输出到其他文本文件
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


missing data在上面是被表示为空字符串，但是可能并不容易看出来。
我们也许希望它是用其他字符表示到，比如用 NULL 来表示

In [8]:
import pandas as pd
import sys

data = pd.read_csv('chapter06/ex5.csv')
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [10]:
import pandas as pd
import sys

data = pd.read_csv('chapter06/ex5.csv')
data.to_csv(sys.stdout, na_rep='NAN')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NAN
1,two,5,6,NAN,8,world
2,three,9,10,11.0,12,foo


观察上面到几个例子，我们可以看出来，默认请看下，index和header都会被输出。
不过，我们也可以禁止输出他们

In [12]:
import pandas as pd
import sys

data = pd.read_csv('chapter06/ex5.csv')
data.to_csv(sys.stdout, na_rep='NAN', index=False, header=False)

one,1,2,3.0,4,NAN
two,5,6,NAN,8,world
three,9,10,11.0,12,foo


此外我们还可以指定输出某些列

In [17]:
import pandas as pd
import sys

data = pd.read_csv('chapter06/ex5.csv')
data.to_csv(sys.stdout, na_rep='NAN', columns=['a', 'b','c'], index=False)

a,b,c
1,2,3.0
5,6,NAN
9,10,11.0


我们也能将Series输出到文本：

In [19]:
import pandas as pd
import numpy as np

dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)

In [20]:
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int64

In [21]:
ts.to_csv('chapter06/tseries.csv')

In [22]:
!cat chapter06/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


从csv转为series的方法一：先化为DataFrame，然后利用loc方法转为Series

In [86]:
result = pd.read_csv('chapter06/tseries.csv', parse_dates=True, header=None,index_col=0)

In [82]:
result

,1
0,
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [73]:
x = result.loc[:,1]

In [75]:
x

0
2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Name: 1, dtype: int64

方法二：这也是原书第一版中的方法，不过这个方法已经被弃用。

In [83]:
df = pd.Series.from_csv('chapter06/tseries.csv')  
# Series.from_csv has DEPRECATED. That's whi I use read_csv in above cells.

/Users/ywfang/miniconda3/envs/plotenv/lib/python3.6/site-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [84]:
df

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

方法三：使用 squeeze=True，这是我个人最推荐的方法。不过根据官方的文档说法，squeeze只在原始数据包含一个columns时候才会返回Series

squeeze : boolean, default False

If the parsed data only contains one column then return a Series

In [92]:
result = pd.read_csv('chapter06/tseries.csv', parse_dates=True, header=None,index_col=0,squeeze=True)

In [89]:
type(result)


pandas.core.series.Series

In [93]:
result


0
2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Name: 1, dtype: int64

### Working with delimited formats

Although we can use functions like pandas.read_table to laod most formas of tabular data, however, munal processing may be necessary sometimes. 其实实际情况中，我们拿到的数据很有可能会有一些很奇特的行，他们无法被read_table等函数识别导入。

为了说明这些基本工具，我们来看一些简单的例子

In [1]:
!cat chapter06/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"
